In [1]:
!pip install pygame

    100% |████████████████████████████████| 11.4MB 4.1MB/s eta 0:00:01
fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
from gamestate import GameState, start_ai_only_game
from models.RandomModel import RandomModel
from models.HeuristicModel import HeuristicModel
from models.DeepRLModel import DeepRLModel, DQN, STATE_ENCODING_LENGTH
from trainutil import train_models, test_models, get_epsilon_decay_factor
import fsutils as fs

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import torch
torch.cuda.is_available()

True

In [9]:
episodes = 150
steps = 1000 

gamma = 0.99

buffer_capacity = 15000
buffer_learn_thresh = 0.5
batch_size = 128

target_update = 10

start_epsilon = 0.99
min_epsilon = 0.2
decay_window = 100
decay = get_epsilon_decay_factor(start_epsilon, min_epsilon, decay_window)


calc_decay_window = episodes - int(buffer_learn_thresh * buffer_capacity / steps)
print("decay_window calc:", calc_decay_window)
print("Learning starts at episode:", int(buffer_learn_thresh * buffer_capacity / steps))
print("Decay rate: ", decay)

decay_window calc: 143
Learning starts at episode: 7
Decay rate:  0.9841333470999254


In [8]:
net = DQN(STATE_ENCODING_LENGTH, 8)
fs.load_net_from_device(net, "drl_fc32", "cpu")

DQN(
  (fc1): Linear(in_features=9, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=8, bias=True)
  (relu): ReLU()
)

In [10]:
env = GameState()
deep_rl_model = DeepRLModel(
                    epsilon = start_epsilon,
                    min_epsilon = min_epsilon,
                    epsilon_decay = decay,
                    gamma = gamma,
                    buffer_capacity = buffer_capacity,
                    replay_buffer_learn_thresh = buffer_learn_thresh,
                    batch_size = batch_size,
                    lr = 1e-3,
                    model = None)

# rand_model_1 = RandomModel(min_steps=5, max_steps=10)
# rand_model_2 = RandomModel(min_steps=5, max_steps=10)
models = [deep_rl_model]

print(deep_rl_model.model)

DQN(
  (fc1): Linear(in_features=9, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=8, bias=True)
  (relu): ReLU()
)


In [ ]:
model_name = "drl_fc32_7"
train_models(env, models, episodes=episodes, steps=steps, target_update=target_update, print_every=1000, model_name=model_name)
# test_models(env, models, steps=2000)


TRAIN MODE
=== Starting Episode 0 ===


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2920: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Mean Episode Loss: nan | Episode Reward: -6.9396 | Mean Reward: -6.9396
=== Starting Episode 1 ===
Mean Episode Loss: nan | Episode Reward: 0.4925 | Mean Reward: -3.2235
=== Starting Episode 2 ===
Mean Episode Loss: nan | Episode Reward: 4.3817 | Mean Reward: -0.6885
=== Starting Episode 3 ===
Mean Episode Loss: nan | Episode Reward: 3.4538 | Mean Reward: 0.3471
=== Starting Episode 4 ===
Mean Episode Loss: nan | Episode Reward: -6.0141 | Mean Reward: -0.9251
=== Starting Episode 5 ===
Mean Episode Loss: nan | Episode Reward: -11.3456 | Mean Reward: -2.6619
=== Starting Episode 6 ===
Mean Episode Loss: nan | Episode Reward: -2.7720 | Mean Reward: -2.6776
=== Starting Episode 7 ===
----LEARNING BEGINS----
Mean Episode Loss: 0.0572 | Episode Reward: -6.8859 | Mean Reward: -3.2037
=== Starting Episode 8 ===
Mean Episode Loss: 0.0098 | Episode Reward: -8.3747 | Mean Reward: -3.7782
=== Starting Episode 9 ===
Mean Episode Loss: 0.0090 | Episode Reward: 5.2993 | Mean Reward: -2.8705
=== Star

In [ ]:
fs.save_net_to_disk(deep_rl_model.model, model_name)